In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [7]:

class Constants():
    ''' The Constants for the mapping terms to index '''
    PAD = 0
    UNK = 1
    SOS = 2
    EOS = 3

    PAD_WORD = '<PAD>'
    UNK_WORD = '<UNK>'
    SOS_WORD = '<SOS>'
    EOS_WORD = '<EOS>'

In [1]:

class BaselineLSTMModel(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int,  lstm_hidden: int, droup_out: float, num_lstm_layers: int):
        super(BaselineLSTMModel, self).__init__()

        self.emb = nn.Embedding(vocab_size, embedding_dim, Constants.PAD)
        self.lstm_1 = nn.LSTM(embedding_dim, lstm_hidden, batch_first=True, dropout= dropout, num_layers= num_lstm_layers)

        self.output_net = nn.Sequential(
            nn.Linear(lstm_hidden, vocab_size),
            nn.Softmax()
        )

        ## The trainable init h0 and c0 in LSTM.
        self.h0 = nn.Parameter(torch.randn(1, lstm_hidden))
        self.c0 = nn.Parameter(torch.randn(1, lstm_hidden))

NameError: name 'nn' is not defined

In [7]:
emb_dim: int  = 256
batch_size = 16
vocab_size = 40
seq_len = 20
lstm_hidden = 32
lstm_layer=2 

fake_data = torch.randint(0, 10, (batch_size, seq_len))

In [8]:
h0 = nn.Parameter(torch.randn(lstm_layer,1, lstm_hidden))
c0 = nn.Parameter(torch.randn(lstm_layer,1, lstm_hidden))

In [9]:
emb_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim= emb_dim)

In [10]:
emb_out = emb_layer(fake_data)

In [12]:
fake_data.shape # (B,S)

torch.Size([16, 20])

In [13]:
emb_out.shape #(B, S, F)

torch.Size([16, 20, 256])

In [11]:
lstm = nn.LSTM(emb_dim, lstm_hidden, batch_first=True, dropout= 0.8, num_layers= 2) # expect  (batch, seq, feature)

In [14]:
lstm_out, lstm_h =  lstm(emb_out, (h0.repeat(1, batch_size, 1) ,c0.repeat(1, batch_size, 1)))

In [19]:
lstm_out.size() # (B, S, F)

torch.Size([16, 20, 32])

In [20]:
lstm_out[:, -1, :].shape

torch.Size([16, 32])

In [21]:
output_net = nn.Linear(lstm_hidden, vocab_size)

In [22]:
out = output_net(lstm_out)

In [23]:
out.shape # (B, S, F)

torch.Size([16, 20, 40])

In [30]:
lossCal = nn.CrossEntropyLoss( )

In [31]:
final_loss = lossCal(out[:, :-1,:], fake_data[:, 1:])

ValueError: Expected target size (16, 40), got torch.Size([16, 19])

In [110]:
final_loss = lossCal(out[:, :-1,:].permute(0, 2,1 ), fake_data[:, 1:])

In [33]:
out[:, :-1,:].transpose(2,1 ).shape

torch.Size([16, 40, 19])

In [45]:
out.shape

torch.Size([16, 20, 40])

In [46]:
softmax_out = F.softmax(out, dim= -1)

In [48]:
softmax_out[1, 1, :].sum()

tensor(1., grad_fn=<SumBackward0>)

In [49]:
argmax_out = torch.argmax(softmax_out, dim= -1)

In [50]:
argmax_out.shape

torch.Size([16, 20])

In [55]:
softmax_out.shape

torch.Size([16, 20, 40])

In [61]:
torch.index_select(argmax_out, dim= 0, index= lens ).shape

torch.Size([16, 20])

In [60]:
argmax_out.shape

torch.Size([16, 20])

In [63]:
argmax_out[lens].shape

torch.Size([16, 20])

In [64]:
argmax_out.shape

torch.Size([16, 20])

In [67]:
torch.BoolTensor(batch_size, vocab_size).shape

torch.Size([16, 40])

In [72]:
F.one_hot(lens, argmax_out.shape[1]).shape

torch.Size([16, 20])

In [78]:
final_out_mask = torch.gt(F.one_hot(lens, argmax_out.shape[1]), 0)

In [92]:
argmax_out.masked_select(final_out_mask)

tensor([36,  5,  2, 15,  2, 26, 15,  6, 26, 26, 26, 15,  7, 26,  2, 26])

In [90]:
argmax_out[3, :]

tensor([26, 26, 26, 18, 15, 36, 15, 15, 13, 13, 13, 16, 13, 16, 26, 25, 25,  2,
         2,  2])

In [58]:
lens = torch.tensor([1,2,3,4,5,6,7,8,1,2,3,4,2,1,2,3,])

In [25]:
fake_data.shape

torch.Size([16, 20])

In [27]:
out[:, :-1,:].shape

torch.Size([16, 19, 40])

In [29]:
fake_data[:, 1:].shape

torch.Size([16, 19])

In [112]:
final_loss.backward()

In [107]:
fake_data[:, 1:].shape

torch.Size([16, 19])

In [106]:
seq_len

20